In [1]:
# ============================================================================
# SILVER LAYER - COMPREHENSIVE DATA CLEANING
# ============================================================================

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime

print("="*80)
print("SILVER LAYER TRANSFORMATIONS STARTING")
print("="*80)
print(f"Timestamp: {datetime.now()}")

# Bronze tablolarini yukle
print("\nLoading Bronze tables...")

df_batch = spark.table("bronze_lakehouse.bronze_hotel_bookings")
df_stream = spark.table("bronze_lakehouse.bronze_hotel_bookings_stream")
df_weather = spark.table("bronze_lakehouse.bronze_weather_data")
df_currency = spark.table("bronze_lakehouse.bronze_currency_rates")

print(f"Batch records: {df_batch.count():,}")
print(f"Stream records: {df_stream.count():,}")
print(f"Weather records: {df_weather.count():,}")
print(f"Currency records: {df_currency.count():,}")

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 3, Finished, Available, Finished)

SILVER LAYER TRANSFORMATIONS STARTING
Timestamp: 2025-11-29 16:13:22.453513

Loading Bronze tables...
Batch records: 1,363,403
Stream records: 450,000
Weather records: 9
Currency records: 166


In [2]:
# ============================================================================
# BATCH + STREAM UNION
# ============================================================================

print("\n" + "="*80)
print("COMBINING BATCH + STREAM DATA")
print("="*80)

# Stream tablosunda 'layer' kolonu yoksa ekle
if 'layer' not in df_stream.columns:
    df_stream = df_stream.withColumn("layer", lit("bronze_stream"))

# Union (ayni kolonlara sahip olmalı)
df_combined = df_batch.unionByName(df_stream, allowMissingColumns=True)

print(f"Total combined records: {df_combined.count():,}")

# Source ayirimi icin flag ekle
df_combined = df_combined.withColumn(
    "data_source",
    when(col("layer") == "bronze", "batch")
    .when(col("layer") == "bronze_stream", "stream")
    .otherwise("unknown")
)

print("\nData source distribution:")
df_combined.groupBy("data_source").count().show()

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 4, Finished, Available, Finished)


COMBINING BATCH + STREAM DATA
Total combined records: 1,813,403

Data source distribution:
+-----------+-------+
|data_source|  count|
+-----------+-------+
|      batch|1363403|
|     stream| 450000|
+-----------+-------+



In [3]:
# ============================================================================
# DATA TYPE CONVERSIONS
# ============================================================================

print("\n" + "="*80)
print("DATA TYPE TRANSFORMATIONS")
print("="*80)

# Sayisal kolonlari integer/double'a cevir
numeric_columns = [
    'star_rating', 'total_rooms', 'nights', 'adults', 'children', 
    'infants', 'rooms_booked', 'total_price', 'room_price', 
    'tax_amount', 'service_fee', 'paid_amount', 'discount_amount',
    'helpful_votes', 'review_rating'
]

df_typed = df_combined

for col_name in numeric_columns:
    if col_name in df_typed.columns:
        # Gecersiz degerleri temizle
        df_typed = df_typed.withColumn(
            col_name,
            when(
                col(col_name).rlike('^[0-9]+\.?[0-9]*$'),  # Sadece sayilar
                col(col_name).cast(DoubleType())
            ).otherwise(None)
        )

print("Numeric columns converted")

# Boolean kolonlari
boolean_columns = ['is_cancelled', 'marketing_consent', 'is_verified_review']

for col_name in boolean_columns:
    if col_name in df_typed.columns:
        df_typed = df_typed.withColumn(
            col_name,
            when(col(col_name).isin(['True', 'true', '1', 't', 'T']), True)
            .when(col(col_name).isin(['False', 'false', '0', 'f', 'F']), False)
            .otherwise(None)
        )

print("Boolean columns converted")

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 5, Finished, Available, Finished)


DATA TYPE TRANSFORMATIONS
Numeric columns converted
Boolean columns converted


In [4]:
# ============================================================================
# PLACEHOLDER & NULL VALUE CLEANING
# ============================================================================

print("\n" + "="*80)
print("CLEANING PLACEHOLDERS AND NULLS")
print("="*80)

# Placeholder degerler
placeholders = ['___', '???', '--', '...', 'NULL', '!!', 'null', 'N/A', 'NA']

# Tum string kolonlarda placeholder'lari null yap
string_columns = [f.name for f in df_typed.schema.fields if isinstance(f.dataType, StringType)]

df_clean = df_typed

for col_name in string_columns:
    df_clean = df_clean.withColumn(
        col_name,
        when(
            (col(col_name).isin(placeholders)) | 
            (trim(col(col_name)) == '') |
            (col(col_name).startswith('!!')),
            None
        ).otherwise(trim(col(col_name)))
    )

print(f"Cleaned {len(string_columns)} string columns")

# Sayisal sutunlarda gecersiz degerler (9999, -1, 0 gibi)
df_clean = df_clean.withColumn(
    "total_price",
    when((col("total_price") == 9999) | (col("total_price") < 0), None)
    .otherwise(col("total_price"))
)

df_clean = df_clean.withColumn(
    "room_price",
    when((col("room_price") == 9999) | (col("room_price") == 0), None)
    .otherwise(col("room_price"))
)

df_clean = df_clean.withColumn(
    "paid_amount",
    when(col("paid_amount") < 0, None)
    .otherwise(col("paid_amount"))
)

print("Invalid numeric values cleaned")

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 6, Finished, Available, Finished)


CLEANING PLACEHOLDERS AND NULLS
Cleaned 60 string columns
Invalid numeric values cleaned


In [5]:
# ============================================================================
# DATE CLEANING & STANDARDIZATION
# ============================================================================

print("\n" + "="*80)
print("DATE CLEANING")
print("="*80)

# Tarih kolonlari
date_columns = [
    'birth_date', 'registration_date', 'booking_date', 
    'checkin_date', 'checkout_date', 'cancellation_date', 
    'review_date', 'stay_date'
]

def clean_date_column(df, col_name):
    """Tarih kolonunu temizle ve standartlastir"""
    
    # Gecersiz tarih formatlari
    invalid_dates = ['0000-00-00', 'Jan 32', '02/30/', '13-']
    
    # Temizle
    df_result = df.withColumn(
        f"{col_name}_clean",
        when(
            col(col_name).isNull() | 
            (col(col_name).rlike('0000-00-00')) |
            (col(col_name).contains('Jan 32')) |
            (col(col_name).contains('13-')) |
            (col(col_name).contains('02/30')),
            None
        ).otherwise(col(col_name))
    )
    
    # Farkli formatlardan parse et
    df_result = df_result.withColumn(
        f"{col_name}_parsed",
        coalesce(
            to_date(col(f"{col_name}_clean"), "yyyy-MM-dd"),
            to_date(col(f"{col_name}_clean"), "MM/dd/yyyy"),
            to_date(col(f"{col_name}_clean"), "dd/MM/yyyy"),
            to_date(col(f"{col_name}_clean"), "yyyy/MM/dd")
        )
    )
    
    # Orijinal kolonu kaldir, yenisini koy
    df_result = df_result.drop(col_name, f"{col_name}_clean") \
                         .withColumnRenamed(f"{col_name}_parsed", col_name)
    
    return df_result

# Tum tarih kolonlarini temizle
df_clean_dates = df_clean

for date_col in date_columns:
    if date_col in df_clean_dates.columns:
        print(f"Cleaning {date_col}...")
        df_clean_dates = clean_date_column(df_clean_dates, date_col)

print("All date columns cleaned and standardized")

# Timestamp kolonlari
timestamp_columns = ['created_timestamp', 'updated_timestamp', 'created_at', 'updated_at']

for ts_col in timestamp_columns:
    if ts_col in df_clean_dates.columns:
        df_clean_dates = df_clean_dates.withColumn(
            ts_col,
            to_timestamp(col(ts_col))
        )

print("Timestamp columns converted")

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 7, Finished, Available, Finished)


DATE CLEANING
Cleaning birth_date...
Cleaning registration_date...
Cleaning booking_date...
Cleaning checkin_date...
Cleaning checkout_date...
Cleaning cancellation_date...
Cleaning review_date...
Cleaning stay_date...
All date columns cleaned and standardized
Timestamp columns converted


In [6]:
# ============================================================================
# TEXT NORMALIZATION
# ============================================================================

print("\n" + "="*80)
print("TEXT NORMALIZATION")
print("="*80)

# Buyuk/kucuk harf duzenleme
capitalized_columns = ['hotel_name', 'city', 'country', 'first_name', 'last_name']

for col_name in capitalized_columns:
    if col_name in df_clean_dates.columns:
        df_clean_dates = df_clean_dates.withColumn(
            col_name,
            initcap(col(col_name))  # Her kelimenin ilk harfi buyuk
        )

print("Capitalized columns normalized")

# Email kucuk harf
if 'email' in df_clean_dates.columns:
    df_clean_dates = df_clean_dates.withColumn(
        "email",
        lower(trim(col("email")))
    )
    print("Email normalized")

# Kategorik kolonlari standartlastir
df_clean_dates = df_clean_dates.withColumn(
    "loyalty_level_customer",
    when(col("loyalty_level_customer").isNotNull(),
         upper(col("loyalty_level_customer")))
    .otherwise(None)
)

df_clean_dates = df_clean_dates.withColumn(
    "payment_status",
    when(col("payment_status").isNotNull(),
         initcap(col("payment_status")))
    .otherwise(None)
)

print("Categorical columns standardized")

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 8, Finished, Available, Finished)


TEXT NORMALIZATION
Capitalized columns normalized
Email normalized
Categorical columns standardized


In [7]:
# ============================================================================
# DUPLICATE REMOVAL
# ============================================================================

print("\n" + "="*80)
print("REMOVING DUPLICATES")
print("="*80)

initial_count = df_clean_dates.count()
print(f"Records before deduplication: {initial_count:,}")

# booking_id bazinda deduplicate
df_dedup = df_clean_dates.dropDuplicates(['booking_id'])

final_count = df_dedup.count()
duplicates_removed = initial_count - final_count

print(f"Records after deduplication: {final_count:,}")
print(f"Duplicates removed: {duplicates_removed:,}")

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 9, Finished, Available, Finished)


REMOVING DUPLICATES
Records before deduplication: 1,813,403
Records after deduplication: 998,885
Duplicates removed: 814,518


In [8]:
# ============================================================================
# DERIVED COLUMNS
# ============================================================================

print("\n" + "="*80)
print("CREATING DERIVED COLUMNS")
print("="*80)

# Total guests
df_enriched = df_dedup.withColumn(
    "total_guests",
    coalesce(col("adults"), lit(0)) + 
    coalesce(col("children"), lit(0)) + 
    coalesce(col("infants"), lit(0))
)

# Booking to checkin days
df_enriched = df_enriched.withColumn(
    "booking_lead_days",
    datediff(col("checkin_date"), col("booking_date"))
)

# Length of stay (nights hesabi - alternatif)
df_enriched = df_enriched.withColumn(
    "calculated_nights",
    datediff(col("checkout_date"), col("checkin_date"))
)

# Revenue per night
df_enriched = df_enriched.withColumn(
    "revenue_per_night",
    when(col("nights") > 0, col("total_price") / col("nights"))
    .otherwise(None)
)

# Age calculation (dogum tarihinden)
df_enriched = df_enriched.withColumn(
    "customer_age",
    when(col("birth_date").isNotNull(),
         floor(datediff(current_date(), col("birth_date")) / 365))
    .otherwise(None)
)

# Booking year/month
df_enriched = df_enriched.withColumn("booking_year", year(col("booking_date")))
df_enriched = df_enriched.withColumn("booking_month", month(col("booking_date")))
df_enriched = df_enriched.withColumn("booking_quarter", quarter(col("booking_date")))

# Season
df_enriched = df_enriched.withColumn(
    "booking_season",
    when(col("booking_month").isin([12, 1, 2]), "Winter")
    .when(col("booking_month").isin([3, 4, 5]), "Spring")
    .when(col("booking_month").isin([6, 7, 8]), "Summer")
    .when(col("booking_month").isin([9, 10, 11]), "Fall")
    .otherwise(None)
)

print("Derived columns created")

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 10, Finished, Available, Finished)


CREATING DERIVED COLUMNS
Derived columns created


In [9]:
# ============================================================================
# SILVER TABLE CREATION (WITH TIMESTAMP FIX)
# ============================================================================

print("\n" + "="*80)
print("CREATING SILVER TABLE")
print("="*80)

# Spark config - timestamp parsing'i esnek yap
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Timestamp kolonlarini tekrar parse et (daha guvenli)
timestamp_cols = ['created_timestamp', 'updated_timestamp', 'created_at', 'updated_at']

df_silver_fixed = df_enriched

for ts_col in timestamp_cols:
    if ts_col in df_silver_fixed.columns:
        # Once string'e cevir, sonra timestamp yap
        df_silver_fixed = df_silver_fixed.withColumn(
            ts_col,
            when(
                col(ts_col).isNotNull(),
                to_timestamp(col(ts_col).cast("string"), "yyyy-MM-dd HH:mm:ss")
            ).otherwise(None)
        )

print("Timestamp columns fixed")

# Metadata ekle
df_silver = df_silver_fixed.withColumn("silver_processed_timestamp", current_timestamp()) \
                            .withColumn("silver_layer", lit("silver"))

# Silver'a yaz
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("bronze_lakehouse.silver_hotel_bookings")

print("Silver table created: bronze_lakehouse.silver_hotel_bookings")
print(f"Total records: {df_silver.count():,}")

# Onizleme
print("\nSample cleaned data:")
df_silver.select(
    "booking_id", "hotel_name", "city", "country", 
    "booking_date", "total_price", "total_guests",
    "booking_lead_days", "data_source"
).show(5, truncate=False)

print("\nColumn summary:")
print(f"Total columns: {len(df_silver.columns)}")

# Data quality check
print("\nData quality metrics:")
print(f"Records with valid booking_date: {df_silver.filter(col('booking_date').isNotNull()).count():,}")
print(f"Records with valid total_price: {df_silver.filter(col('total_price').isNotNull()).count():,}")
print(f"Records with valid email: {df_silver.filter(col('email').isNotNull()).count():,}")

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 11, Finished, Available, Finished)


CREATING SILVER TABLE
Timestamp columns fixed
Silver table created: bronze_lakehouse.silver_hotel_bookings
Total records: 998,885

Sample cleaned data:
+-------------+--------------------------+---------+-------+------------+-----------+------------+-----------------+-----------+
|booking_id   |hotel_name                |city     |country|booking_date|total_price|total_guests|booking_lead_days|data_source|
+-------------+--------------------------+---------+-------+------------+-----------+------------+-----------------+-----------+
|BKG_000000016|Jackson Inc               |Mombasa  |Kenya  |2024-08-17  |250.08     |3.0         |131              |batch      |
|BKG_000000017|Brock, Nelson And Jackson |Stavanger|Norway |2023-11-06  |1699.77    |5.0         |876              |batch      |
|BKG_000000075|Schwartz, Morales And Reed|Kolkata  |India  |2025-05-23  |1397.58    |9.0         |122              |batch      |
|BKG_000000091|Pena, Craig And Hernandez |Belgrade |Serbia |2025-07-23  |

In [10]:
# ============================================================================
# SILVER WEATHER & CURRENCY TABLES
# ============================================================================

print("\n" + "="*80)
print("CREATING SILVER WEATHER & CURRENCY TABLES")
print("="*80)

# ==================== WEATHER ====================
print("\nProcessing Weather data...")

df_weather_silver = df_weather \
    .withColumn("temperature_fahrenheit", col("temperature_celsius") * 9/5 + 32) \
    .withColumn("wind_speed_mph", col("wind_speed_kmh") * 0.621371) \
    .withColumn("weather_description",
        when(col("weather_code") == 0, "Clear sky")
        .when(col("weather_code") == 1, "Mainly clear")
        .when(col("weather_code") == 2, "Partly cloudy")
        .when(col("weather_code") == 3, "Overcast")
        .when(col("weather_code").between(45, 48), "Foggy")
        .when(col("weather_code").between(51, 67), "Rainy")
        .when(col("weather_code").between(71, 77), "Snowy")
        .when(col("weather_code").between(80, 82), "Rain showers")
        .when(col("weather_code") == 95, "Thunderstorm")
        .otherwise("Unknown")
    ) \
    .withColumn("comfort_level",
        when((col("temperature_celsius") >= 18) & (col("temperature_celsius") <= 26) & 
             (col("humidity_percent") <= 70), "Comfortable")
        .when(col("temperature_celsius") > 30, "Hot")
        .when(col("temperature_celsius") < 10, "Cold")
        .otherwise("Moderate")
    ) \
    .withColumn("silver_processed_timestamp", current_timestamp())

df_weather_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze_lakehouse.silver_weather_data")

print(f"Weather Silver table created: {df_weather_silver.count()} records")

# ==================== CURRENCY ====================
print("\nProcessing Currency data...")

df_currency_silver = df_currency \
    .withColumn("inverse_rate", 1 / col("exchange_rate")) \
    .withColumn("rate_category",
        when(col("exchange_rate") < 1, "Stronger than USD")
        .when(col("exchange_rate") == 1, "Equal to USD")
        .otherwise("Weaker than USD")
    ) \
    .withColumn("major_currency",
        when(col("target_currency").isin(['EUR', 'GBP', 'JPY', 'CHF', 'CNY', 'AUD', 'CAD']), True)
        .otherwise(False)
    ) \
    .withColumn("silver_processed_timestamp", current_timestamp())

df_currency_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze_lakehouse.silver_currency_rates")

print(f"Currency Silver table created: {df_currency_silver.count()} records")

# ==================== SUMMARY ====================
print("\n" + "="*80)
print("ALL SILVER TABLES CREATED")
print("="*80)

# List all Silver tables
silver_tables = spark.sql("SHOW TABLES IN bronze_lakehouse").filter(
    col("tableName").startswith("silver_")
)

print("\nSilver Layer Tables:")
silver_tables.show(truncate=False)

# Statistics
print("\nTable Statistics:")
for table_row in silver_tables.collect():
    table_name = table_row['tableName']
    count = spark.table(f"bronze_lakehouse.{table_name}").count()
    print(f"  {table_name}: {count:,} records")

print("\n" + "="*80)
print("ADIM 5 TAMAMLANDI - SILVER LAYER COMPLETE!")
print("="*80)

StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 12, Finished, Available, Finished)


CREATING SILVER WEATHER & CURRENCY TABLES

Processing Weather data...
Weather Silver table created: 9 records

Processing Currency data...
Currency Silver table created: 166 records

ALL SILVER TABLES CREATED

Silver Layer Tables:
+----------------+---------------------+-----------+
|namespace       |tableName            |isTemporary|
+----------------+---------------------+-----------+
|bronze_lakehouse|silver_hotel_bookings|false      |
|bronze_lakehouse|silver_weather_data  |false      |
|bronze_lakehouse|silver_currency_rates|false      |
+----------------+---------------------+-----------+


Table Statistics:
  silver_hotel_bookings: 998,885 records
  silver_weather_data: 9 records
  silver_currency_rates: 166 records

ADIM 5 TAMAMLANDI - SILVER LAYER COMPLETE!


In [11]:
# Fabric Notebook: Silver Layer Transformations
# Purpose: Clean and transform Bronze hotel bookings to Silver layer

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, trim, upper, lower, regexp_replace, when, 
    coalesce, lit, current_timestamp, avg, stddev
)
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions import abs as spark_abs

print("=" * 80)
print("🧹 SILVER LAYER TRANSFORMATIONS - HOTEL BOOKINGS CLEANING")
print("=" * 80)

# ============================================================================#
# PART 1: READ BRONZE HOTEL BOOKINGS
# ============================================================================#
print("\n📂 Step 1: Reading Bronze hotel bookings...")
df_bronze = spark.read.table("silver_hotel_bookings")

bronze_count = df_bronze.count()
print(f"✅ Bronze hotel bookings: {bronze_count:,} records")

# ============================================================================#
# PART 2: NULL VALUE HANDLING
# ============================================================================#
print("\n" + "=" * 80)
print("🔍 TASK 1: NULL VALUE HANDLING")
print("=" * 80)

critical_columns = ['booking_id', 'hotel_id', 'customer_id', 'booking_date']

null_count_before = df_bronze.filter(
    col('booking_id').isNull() | 
    col('hotel_id').isNull() | 
    col('customer_id').isNull()
).count()

print(f"\n⚠️  Records with NULL in critical columns: {null_count_before:,}")

df_clean = df_bronze.filter(
    col('booking_id').isNotNull() & 
    col('hotel_id').isNotNull() & 
    col('customer_id').isNotNull() &
    col('booking_date').isNotNull()
)

# Fill nulls in non-critical columns with defaults
df_clean = df_clean \
    .fillna({'special_requests': 'No requests'}) \
    .fillna({'promotion_code': 'NO_PROMO'}) \
    .fillna({'cancellation_reason': 'Not Cancelled'}) \
    .fillna({'review_title': 'No Review'}) \
    .fillna({'review_text': 'No Review'})

clean_count = df_clean.count()
removed = bronze_count - clean_count
print(f"✅ Records removed (NULL critical): {removed:,}")
print(f"✅ Clean records remaining: {clean_count:,}")

# ============================================================================#
# PART 3: REMOVE DUPLICATES
# ============================================================================#
print("\n" + "=" * 80)
print("🔍 TASK 2: REMOVE DUPLICATES")
print("=" * 80)

duplicate_count = df_clean.count() - df_clean.select('booking_id').distinct().count()
print(f"\n⚠️  Duplicate booking_ids found: {duplicate_count:,}")

df_clean = df_clean.dropDuplicates(['booking_id'])
print(f"✅ Records after deduplication: {df_clean.count():,}")

# ============================================================================#
# PART 4: FIX DATA TYPES
# ============================================================================#
print("\n" + "=" * 80)
print("🔍 TASK 3: FIX DATA TYPES")
print("=" * 80)

df_clean = df_clean \
    .withColumn("star_rating", col("star_rating").cast(IntegerType())) \
    .withColumn("total_rooms", col("total_rooms").cast(IntegerType())) \
    .withColumn("nights", col("nights").cast(IntegerType())) \
    .withColumn("adults", col("adults").cast(IntegerType())) \
    .withColumn("children", col("children").cast(IntegerType())) \
    .withColumn("infants", col("infants").cast(IntegerType())) \
    .withColumn("rooms_booked", col("rooms_booked").cast(IntegerType())) \
    .withColumn("total_price", col("total_price").cast(DoubleType())) \
    .withColumn("room_price", col("room_price").cast(DoubleType())) \
    .withColumn("tax_amount", col("tax_amount").cast(DoubleType())) \
    .withColumn("service_fee", col("service_fee").cast(DoubleType())) \
    .withColumn("paid_amount", col("paid_amount").cast(DoubleType())) \
    .withColumn("discount_amount", col("discount_amount").cast(DoubleType())) \
    .withColumn("review_rating", col("review_rating").cast(IntegerType())) \
    .withColumn("helpful_votes", col("helpful_votes").cast(IntegerType())) \
    .withColumn("latitude", col("latitude").cast(DoubleType())) \
    .withColumn("longitude", col("longitude").cast(DoubleType()))

print("✅ Numeric columns converted")

# ============================================================================#
# PART 5: STANDARDIZE DATE FORMATS
# ============================================================================#
print("\n" + "=" * 80)
print("🔍 TASK 4: STANDARDIZE DATE FORMATS")
print("=" * 80)

df_clean = df_clean.withColumn(
    "date_validation_flag",
    when(
        (col("checkin_date").isNotNull()) & 
        (col("checkout_date").isNotNull()) & 
        (col("checkout_date") <= col("checkin_date")),
        "INVALID"
    ).otherwise("VALID")
)

invalid_dates = df_clean.filter(col("date_validation_flag") == "INVALID").count()
print(f"⚠️  Invalid date sequences found: {invalid_dates:,}")

df_clean = df_clean.filter(col("date_validation_flag") == "VALID")
print(f"✅ Records after date validation: {df_clean.count():,}")

# ============================================================================#
# PART 6: FIX TEXT ISSUES
# ============================================================================#
print("\n" + "=" * 80)
print("🔍 TASK 5: FIX TEXT ISSUES")
print("=" * 80)

text_columns = ['hotel_name', 'city', 'country', 'hotel_type', 
                'first_name', 'last_name', 'city_customer', 'country_customer']

for text_col in text_columns:
    if text_col in df_clean.columns:
        df_clean = df_clean \
            .withColumn(text_col, trim(col(text_col))) \
            .withColumn(text_col, regexp_replace(col(text_col), r'[\s]+', ' ')) \
            .withColumn(text_col, regexp_replace(col(text_col), r'[_]{2,}', 'Unknown')) \
            .withColumn(text_col, when(col(text_col) == '', None).otherwise(col(text_col))) \
            .withColumn(text_col, when(col(text_col) == '--', None).otherwise(col(text_col))) \
            .withColumn(text_col, when(col(text_col) == '???', None).otherwise(col(text_col)))

df_clean = df_clean.withColumn("email", lower(trim(col("email"))))
print("✅ Text columns cleaned")

# ============================================================================#
# PART 7: REMOVE/FIX INCORRECT VALUES
# ============================================================================#
print("\n" + "=" * 80)
print("🔍 TASK 6: REMOVE/FIX INCORRECT VALUES")
print("=" * 80)

df_clean = df_clean.withColumn(
    "star_rating",
    when((col("star_rating") < 1) | (col("star_rating") > 5), None)
    .otherwise(col("star_rating"))
)

numeric_cols_positive = ['nights', 'adults', 'rooms_booked', 'total_price', 'room_price', 'paid_amount']

for num_col in numeric_cols_positive:
    df_clean = df_clean.withColumn(
        num_col,
        when(col(num_col) < 0, spark_abs(col(num_col)))
        .otherwise(col(num_col))
    )
from pyspark.sql.functions import col, when

df_clean = df_clean.withColumn(
    "is_cancelled",
    when(col("is_cancelled").cast("string").isin("True", "true", "TRUE", "1"), True)
    .when(col("is_cancelled").cast("string").isin("False", "false", "FALSE", "0"), False)
    .otherwise(False)
)


print("✅ Incorrect values fixed")

# ============================================================================#
# PART 8: ADD METADATA AND WRITE TO SILVER
# ============================================================================#
print("\n" + "=" * 80)
print("💾 WRITING TO SILVER LAYER")
print("=" * 80)

df_silver = df_clean \
    .withColumn("silver_ingestion_timestamp", current_timestamp()) \
    .withColumn("silver_version", lit("v1.0"))

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_hotel_bookings")

silver_count = spark.read.table("silver_hotel_bookings").count()
print(f"✅ Silver table updated: {silver_count:,} records")

print("\n" + "=" * 80)
print("✅ HOTEL BOOKINGS CLEANING COMPLETED!")
print("=" * 80)
print(f"\n📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 13, Finished, Available, Finished)

🧹 SILVER LAYER TRANSFORMATIONS - HOTEL BOOKINGS CLEANING

📂 Step 1: Reading Bronze hotel bookings...
✅ Bronze hotel bookings: 998,885 records

🔍 TASK 1: NULL VALUE HANDLING

⚠️  Records with NULL in critical columns: 57,384
✅ Records removed (NULL critical): 98,492
✅ Clean records remaining: 900,393

🔍 TASK 2: REMOVE DUPLICATES

⚠️  Duplicate booking_ids found: 0
✅ Records after deduplication: 900,393

🔍 TASK 3: FIX DATA TYPES
✅ Numeric columns converted

🔍 TASK 4: STANDARDIZE DATE FORMATS
⚠️  Invalid date sequences found: 60,796
✅ Records after date validation: 839,597

🔍 TASK 5: FIX TEXT ISSUES
✅ Text columns cleaned

🔍 TASK 6: REMOVE/FIX INCORRECT VALUES
✅ Incorrect values fixed

💾 WRITING TO SILVER LAYER
✅ Silver table updated: 839,597 records

✅ HOTEL BOOKINGS CLEANING COMPLETED!

📅 Timestamp: 2025-11-29 16:17:47


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, trim, when
from pyspark.sql.types import StringType, IntegerType, DoubleType
from pyspark.sql import functions as F

# Spark session (zaten varsa bu satır atlanabilir)
spark = SparkSession.builder.getOrCreate()

# 1. Silver tablosundan veri oku
df = spark.read.table("silver_hotel_bookings")

# 2. Key kolonlardaki NULL değerleri temizle
key_columns = ["hotel_id", "customer_id", "booking_id"]
df = df.dropna(subset=key_columns)

# 3. Duplicates temizle
df = df.dropDuplicates()

# 4. Veri tiplerini tabloya göre düzelt (string, int, double)
string_cols = ["hotel_id","hotel_name","country","city","hotel_type","hotel_facilities","hotel_description",
               "nearby_attractions","website","customer_id","first_name","last_name","full_name","email",
               "phone","birth_date","gender","country_customer","city_customer","address","postal_code",
               "language_preference","loyalty_level_customer","registration_date","marketing_consent",
               "booking_id","booking_channel","special_requests","cancellation_date","cancellation_reason",
               "payment_status","payment_method","booking_source","promotion_code","created_timestamp",
               "updated_timestamp","created_by","booking_status","review_id","review_title","review_text",
               "reviewer_location","stay_date","room_type_reviewed","source_system","created_at","updated_at"]

int_cols = ["star_rating","total_rooms","nights","adults","children","infants","rooms_booked","review_rating","helpful_votes"]
double_cols = ["latitude","longitude","total_price","room_price","tax_amount","service_fee","paid_amount","discount_amount"]

for c in string_cols:
    if c in df.columns:
        df = df.withColumn(c, col(c).cast(StringType()))
for c in int_cols:
    if c in df.columns:
        df = df.withColumn(c, col(c).cast(IntegerType()))
for c in double_cols:
    if c in df.columns:
        df = df.withColumn(c, col(c).cast(DoubleType()))

# 5. Tarih kolonlarını string olarak bırak (overwrite hatasını önlemek için)
date_cols = ["birth_date","registration_date","booking_date","checkin_date","checkout_date",
             "cancellation_date","review_date","stay_date","created_timestamp","updated_timestamp","created_at","updated_at"]
for c in date_cols:
    if c in df.columns:
        df = df.withColumn(c, col(c).cast(StringType()))

# 6. Metin kolonlarını temizle (trim, lowercase örnek)
text_cols = ["hotel_name","hotel_type","hotel_facilities","hotel_description","nearby_attractions",
             "first_name","last_name","full_name","email","address","city","country"]
for c in text_cols:
    if c in df.columns:
        df = df.withColumn(c, trim(lower(col(c))))

# 7. Yanlış veya eksik değerleri düzelt (örnek: gender, loyalty_level_customer)
df = df.withColumn("gender", when(col("gender").isin("M","F"), col("gender")).otherwise("Other"))
df = df.withColumn("loyalty_level_customer", when(col("loyalty_level_customer").isNotNull(), col("loyalty_level_customer")).otherwise("Bronze"))

# 8. Kategorize veya standartlaştır (örn: hotel_type standardizasyon)
df = df.withColumn("hotel_type", when(col("hotel_type").like("%city%"), "City Hotel")
                                     .when(col("hotel_type").like("%budget%"), "Budget Hotel")
                                     .when(col("hotel_type").like("%business%"), "Business Hotel")
                                     .when(col("hotel_type").like("%resort%"), "Resort")
                                     .otherwise(col("hotel_type")))

# 9. Outlier analizi (basit örnek: star_rating 1-5 dışında ise düzelt)
df = df.withColumn("star_rating", when((col("star_rating")<1) | (col("star_rating")>5), None).otherwise(col("star_rating")))

# 10. Temizlenmiş veriyi silver tablosuna overwrite ile yaz
df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_hotel_bookings")


StatementMeta(, 5ae3ac80-aeef-4c8f-b06c-b5a682ec740b, 16, Finished, Available, Finished)